In [1]:
import os
from nilearn import image as nimg
from nilearn import plotting as nplot
#import matplotilb.pyplot as plt
import numpy as np
import nibabel as nib
#%matplotlib inline


# Setting up for Motion Estimates

In [2]:
import pandas as pd

In [3]:
fmriprep_dir = '/archive/data/SPINS/pipelines/bids_apps/fmriprep/sub-CMH0001/ses-01/func/'

In [4]:
#First we'll load in our data and check the shape
func_file = nimg.load_img("/KIMEL/tigrlab/archive/data/SPINS/pipelines/bids_apps/fmriprep/sub-CMH0001/ses-01/func/sub-CMH0001_ses-01_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")


In [91]:
dummy_TR = 3
func_img = func_file.slicer[:,:,:,dummy_TR:]
img_size = func_img.shape
signal_img = func_img.get_fdata().reshape(-1,img_size[-1]).T

In [6]:
confounds_file = pd.read_csv("/KIMEL/tigrlab/archive/data/SPINS/pipelines/bids_apps/fmriprep/sub-CMH0001/ses-01/func/sub-CMH0001_ses-01_task-rest_run-1_desc-confounds_fixedregressors.tsv", sep = '\t')
confounds_file_drop = confounds_file.loc[dummy_TR:]
confounds_input = confounds_file_drop.values

In [10]:
confounds_file_drop.columns

Index(['global_signal', 'global_signal_derivative1',
       'global_signal_derivative1_power2', 'global_signal_power2', 'std_dvars',
       'dvars', 'framewise_displacement', 't_comp_cor_00', 't_comp_cor_01',
       'cosine00',
       ...
       'a_comp_cor_66_fixed', 'a_comp_cor_67_fixed', 'a_comp_cor_68_fixed',
       'a_comp_cor_69_fixed', 'white_matter_derivative1_fixed',
       'white_matter_power2_fixed', 'white_matter_derivative1_power2_fixed',
       'csf_derivative1_fixed', 'csf_power2_fixed',
       'csf_derivative1_power2_fixed'],
      dtype='object', length=160)

In [7]:
mask_file = nib.load("/KIMEL/tigrlab/archive/data/SPINS/pipelines/bids_apps/fmriprep/sub-CMH0001/ses-01/func/sub-CMH0001_ses-01_task-rest_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz")

**Without Scrubbing**

In [8]:
#Set some constants
high_pass= 0.01
low_pass = 0.1
t_r = 2

#Clean!
clean_img = nimg.clean_img(func_img,
                           confounds=confounds_input,
                           detrend=True,
                           standardize=True,
                           low_pass=low_pass,
                           high_pass=high_pass,
                           t_r=t_r,
                           mask_img = mask_file)


**With Scrubbing**

### Prepare data to create sample_mask

We can use `_optimize_scrub` to generate the sample_mask file.
The `_optimize_scrub` takes three inputs:
1) motion_outlier_index: the vector of indices of TR that exceed the motion threshold
2) n_scans: the number of total TR
3) scrub: the maximum length of the island (scans between two sets of scrubbed TRs) that will be removed

These information can be retrived from the confound file. In SPINS, they are stored in column named `framewise_displacement`.

In [14]:
# Extract FD
all_FD = confounds_file_drop['framewise_displacement']

4      0.103907
5      0.090418
6      0.109402
7      0.061870
8      0.076143
         ...   
207    0.301278
208    0.116285
209    0.114447
210    0.104337
211    0.057574
Name: framewise_displacement, Length: 208, dtype: float64


In [29]:
# Set arguments
## Threshold for motion
FD_threshold = 0.5
## The arguments for _optimize_scrub
motion_outlier_index = np.where(all_FD > FD_threshold)[0]
n_scans = len(all_FD)
scrubs = 5

In [81]:
# Generate the censor_mask
from nilearn.interfaces.fmriprep.load_confounds_scrub import _optimize_scrub
censor_mask = _optimize_scrub(motion_outlier_index, n_scans, scrubs)

# Generate the samplemask
ref_vec = set(range(n_scans))
censor_idx = set(censor_mask)
sample_mask = np.array(sorted(list(ref_vec - censor_idx)))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 18

In [82]:
#Clean with scrubbing!
import nilearn
clean_img = nilearn.signal.clean(signal_img,
                                 confounds=confounds_input,
                                 detrend=True,
                                 standardize=True,
                                 low_pass=low_pass,
                                 high_pass=high_pass,
                                 t_r=t_r,
                                 sample_mask = sample_mask,
                                 filter = 'butterworth')


In [117]:
# Reshape the image file into an array of the original size
clean_img_array = clean_img.T.reshape(*img_size[:-1], len(sample_mask))

In [113]:
# Save it as a Nifti file
clean_img_nifti = nib.Nifti1Image(clean_img_array.astype(np.float64), func_img.affine)
nib.save(clean_img_nifti, f'clean_img_test.nii.gz')